In [ ]:
import numpy as np
from scipy.optimize import fsolve
import pandas as pd
import os
from datetime import datetime

# Glass + air

In [ ]:
# --- Input Parameters ---
params = {
    "q_sun": 1000,            # Incident solar flux (W/m^2)
    "Tr": 0.9148,             # Glass solar weighted transmittance (91.48%)
    "abs": 0.9645,            # Solar absorber's absorgance (96.45%)
    "condition": 'indoor',  
    "T_amb_c": 25,           # Ambient air temperature (Celsius)
    "v": 0.1,                # Wind speed (m/s)
    "a": 0.05,               # Device width (m)
    "b": 0.0035,             # Unit stage thickness (m)
    "t_ins": 0.01,           # Insulation thickness (m)
    "t_glass": 0.003,        # Glass thickness (m)
    "t_air": 0.006,          # air thickness (m)
    "D_a": 3.0e-5,           # Diffusivity of vapor in air (m²/s)
    "k_ins": 0.04,           # Thermal conductivity of insulation (W/mK)
    "k_air": 0.026,          # Thermal conductivity of air (W/mK)
    "k_glass": 1.09,         # Thermal conductivity of glass (W/mK)
    "Pr": 0.71,              # Prandtl number for air 
    "epsilon": 0.95,         # Emissivity of the solar absorber (CNT: 0.95, SSA: 0.03)
    "h_fg": 2357e3,          # Latent heat of vaporization (J/kg, converted from kJ/kg)
    "M_water": 0.018015,     # Molar mass of water (kg/mol)
    "R_gas": 8.314,          # Universal gas constant (J/(mol*K))
    "sigma": 5.67e-8         # Stefan-Boltzmann constant (W/m²K⁴)
}

In [ ]:
# Obtain convective heat transfer coefficient (h_a)    
def get_air_properties(T_amb_c):
    T_amb_k = T_amb_c + 273.15
     # Density using ideal gas law at 1 atm (101325 Pa)
    P_atm = 101325  
    R_air = 287.05  # J/(kg·K) - specific gas constant for air
    rho = P_atm / (R_air * T_amb_k)  # kg/m³

    # Dynamic viscosity using Sutherland's formula
    T_ref = 273.15
    mu_ref = 1.716e-5  # Pa·s at T_ref       
    S = 110.4  # K (Sutherland constant for air)
    mu = mu_ref * (T_amb_k / T_ref)**1.5 * (T_ref + S) / (T_amb_k + S)  # Pa·s
    
    return rho, mu
    
def get_h_a_windy(T_amb_c, v, a):
    T_amb_k = T_amb_c + 273.15
    
    rho, mu = get_air_properties(T_amb_c)
      
    # Calculate Re at length L (device width)
    Re_L = rho * v * a / mu
    if Re_L >= 5e5:
        print(f"Warning: Reynolds number ({Re_L:.2e}) exceeds laminar flow limit (5e5).")
        print(f"The correlation may not be accurate for turbulent flow.")

    # Calculate average convection coefficient 
    h_a = 2 * 0.453 * params["Pr"]**(1/3) * (rho * v / mu)**0.5 * params["k_air"] * a**(-0.5)

    return h_a

def get_h_a_windless(T_amb_c, a):
    T_amb_k = T_amb_c + 273.15
    T_abs_k = 273.15 + 60              # assume 60C for the solar absorber 
    T_avg_k = (T_amb_k + T_abs_k) / 2  # Film temperature for properties

    rho, mu = get_air_properties(T_avg_k - 273.15)

    # Calculate kinematic viscosity
    nu = mu / rho  # m²/s

    # Gravitational acceleration
    g = 9.81  # m/s²

    # Coefficient of volume expansion for ideal gas: β = 1/T
    beta = 1 / T_avg_k  # 1/K
    
    # Characteristic length for horizontal plate
    l_c = a**2/(4*a)
    
    # Calculate Rayleigh number 
    Ra = (g * beta * abs(T_abs_k - T_amb_k) * l_c**3 * params["Pr"]) / (nu**2)

    # Check if Ra is in valid range
    if Ra < 1e4:
        print(f"Warning: Rayleigh number ({Ra:.2e}) is below the correlation range (10^4 to 10^7).")
        print(f"Natural convection may be negligible.")
    elif Ra > 1e7:
        print(f"Warning: Rayleigh number ({Ra:.2e}) exceeds the correlation range (10^4 to 10^7).")
        print(f"The correlation may not be accurate for turbulent natural convection.")

    # Calculate natural convection coefficient (Nu = h_a * k_air/l_c = 0.54 * Ra^0.25)
    h_a = 0.54 * Ra**0.25 * params["k_air"] / l_c

    return h_a

In [ ]:
# --- Load and process spectral data ---

try:
    file_path = os.path.join("data", "Glass_3mm_transmittance.csv")
    
    column_names = ['wavelength_um', 'transmittance_glass', 'transmittance_atm']
    glass_data = pd.read_csv(file_path, skiprows=1, names=column_names)

    for col in column_names:
        glass_data[col] = pd.to_numeric(glass_data[col])

    glass_data['wavelength_m'] = glass_data['wavelength_um'] * 1e-6

    glass_data['delta_lambda'] = glass_data['wavelength_m'].diff().bfill()

    print("Successfully loaded and processed Glass and atmospheric spectral data.")
    
except Exception as e:
    print(f"Error loading or processing Glass_3mm_transmittance.csv: {e}")
    glass_data = None

In [ ]:
# --- Helper Functions ---
def planck_law(wav, T):
    """Calculates spectral radiance using Planck's Law."""
    C1 = 3.7418e-16
    C2 = 1.4388e-2
    return (C1 / (wav**5)) / (np.exp(C2 / (wav * T)) - 1)

def calculate_q_rad(T_f_k, p, spectral_data):
    """
    Calculates q_rad by integrating Planck's law over the spectral data.
    """
    if spectral_data is None: return 0
    T_amb_k = p["T_amb_c"] + 273.15
    
    E_f = planck_law(spectral_data['wavelength_m'], T_f_k)
    E_amb = planck_law(spectral_data['wavelength_m'], T_amb_k)

    tr = spectral_data['transmittance_glass']
    delta_lambda = spectral_data['delta_lambda']
    epsilon = p["epsilon"]

    if p["condition"] == 'indoor':
        integrand = tr * epsilon * (E_f - E_amb) * delta_lambda
        q_rad = integrand.sum()
    else: # outdoor
        trans_atm = spectral_data['transmittance_atm']
        integrand = tr * epsilon * (E_f - (1 - trans_atm) * E_amb) * delta_lambda
        q_rad = integrand.sum()       
    return q_rad
    
def calculate_saturation_concentration(T_k, activity=1.0):
    """
    Calculates water vapor saturation concentration (mol/m³) at a given temperature,
    using August-Roche-Magnus formula, adjusted for the solution's activity.
    """
    T_c = T_k - 273.15
    # P_sat_pure = 610.94 * np.exp((17.625 * T_c) / (T_c + 243.04)) # August-Roche-Magnus formula
    P_sat_pure = 133.32 * 10**8.07131 / 10**(1730.63/(T_c + 233.426)) # Antoine equation
    P_sat_solution = P_sat_pure * activity
    return P_sat_solution / (params["R_gas"] * T_k)


def find_top_surface_temp(T_f_k, p):
    T_amb_k = p["T_amb_c"] + 273.15
    def top_surface_balance(T_top_k):
        q_cond_up = (T_f_k - T_top_k) / ((p["t_glass"] / p["k_glass"]) + (p["t_air"]/p["k_air"]))
        q_conv_out = p["h_a"] * (T_top_k - T_amb_k)
        if p["condition"] == 'indoor':
            q_rad_out = 0.99 * p["sigma"] * (T_top_k**4 - T_amb_k**4)
        if p["condition"] == 'outdoor':
            q_rad_out = (0.0335*(T_top_k-273.15)**2 + 3.0848*(T_top_k-273.15) + 90.8569) * (0.99 / 0.95)
        return q_cond_up - (q_conv_out + q_rad_out)
    initial_guess_T_top = (T_f_k + T_amb_k) / 2
    return fsolve(top_surface_balance, initial_guess_T_top)[0]
    

def solve_stage_backward(q_out, T_b_c, p, is_first_stage=False):
    """
    Solves a single stage backward to find the input heat and front wall temp.
    """
    T_b_k = T_b_c + 273.15

    def find_Tf_from_q_out(T_f_c, q_out_target, T_b_k, p):
        T_f_k = T_f_c + 273.15
        q_cond = p["k_air"] * (T_f_k - T_b_k) / p["b"]
        q_rad_int = p["epsilon"] * p["sigma"]*(T_f_k**4 - T_b_k**4)
        # Use activity=0.98 for evaporating seawater at the front wall
        c_f = calculate_saturation_concentration(T_f_k, activity=0.98)
        # Use activity=1.0 for condensing pure water at the back wall
        c_b = calculate_saturation_concentration(T_b_k, activity=1.0)
        J_evap_molar = p["D_a"] * (c_f - c_b) / p["b"]
        q_evap = (J_evap_molar * p["M_water"]) * p["h_fg"]
        return (q_cond + q_evap + q_rad_int) - q_out_target
        
    initial_guess_T_f = T_b_c + 5.0
    T_f_c_solved = fsolve(find_Tf_from_q_out, initial_guess_T_f, args=(q_out, T_b_k, p))[0]
    T_f_k_solved = T_f_c_solved + 273.15
    
    T_top_k_solved = find_top_surface_temp(T_f_k_solved, p)
    T_top_c_solved = T_top_k_solved - 273.15
    
    T_amb_k = p["T_amb_c"] + 273.15
    R_side = (1 / p["h_a"]) + (p["t_ins"] / p["k_ins"])
    T_avg_side_k = (T_f_k_solved + T_b_k) / 2
    q_side_per_front_area = (T_avg_side_k - T_amb_k) / R_side * (4 * p["b"] / p["a"])

    q_in = q_out + q_side_per_front_area

    if is_first_stage:
        q_rad = calculate_q_rad(T_f_k_solved, p, glass_data)
        R_top = (1 / p["h_a"]) + (p["t_glass"] / p["k_glass"]) + (p["t_air"] / p["k_air"])
        q_cond_up = (T_f_k_solved - T_amb_k) / R_top
        R_top_side = (1 / p["h_a"]) + (p["t_ins"] / p["k_ins"])
        q_cond_side = (((T_f_k_solved + T_amb_k) / 2) - T_amb_k) / R_top_side * (4 * (p["t_air"] + p["t_glass"]) / p["a"])
            

        q_in += q_rad + q_cond_up + q_cond_side
        
    return q_in, T_f_c_solved, T_top_c_solved

In [ ]:
# --- Main Multi-Stage Simulation ---
def run_multistage_simulation(n_stages, params):

    q_out_n = params["q_sun"] * params["abs"] * params["Tr"]
    tolerance = 0.1
    max_iterations = 100
    stage_data = []

    print(f"--- Running Multi-Stage Simulation for {n_stages} Stages ---\n")

    for iteration in range(max_iterations):
        current_stage_data = []
        q_out_current = q_out_n
        T_b_current_c = params["T_b_c"]

        for i in range(n_stages, 0, -1):
            is_first = (i == 1)
            q_in_current, T_f_current_c, T_top_current_c = solve_stage_backward(q_out_current, T_b_current_c, params, is_first_stage=is_first)
            current_stage_data.append({"stage": i, "T_f": T_f_current_c, "T_b": T_b_current_c, "T_top": T_top_current_c})
            q_out_current = q_in_current
            T_b_current_c = T_f_current_c

        current_stage_data.reverse()
        q_in_1_calculated = q_out_current # This is now q_in of the first stage
        error = abs(q_in_1_calculated - params["q_sun"]* params["abs"] * params["Tr"])

        print(f"Iteration {iteration + 1}: Calculated q_in,1 = {q_in_1_calculated:.2f} W/m^2, Error = {error:.2f} W/m^2")

        if error < tolerance:
            print("\nConvergence reached!")
            stage_data = current_stage_data
            break

        scaling_factor = params["q_sun"]* params["abs"] * params["Tr"] / q_in_1_calculated
        q_out_n *= scaling_factor
    else:
        print("\nWarning: Maximum iterations reached without convergence.")
        stage_data = current_stage_data

    # --- Final Calculations and Output ---
    print("\n--- Per-Stage Detailed Results ---")
    total_evap_heat = 0
    for data in stage_data:
        T_f_c = data["T_f"]
        T_b_c = data["T_b"]
        T_f_k = T_f_c + 273.15
        T_b_k = T_b_c + 273.15

        # Calculate vapor flux for this stage 
        c_f = calculate_saturation_concentration(T_f_k, activity=0.98)
        c_b = calculate_saturation_concentration(T_b_k, activity=1.0)
        J_evap_molar = params["D_a"] * (c_f - c_b) / params["b"]
        J_evap_mass = J_evap_molar * params["M_water"]
        J_evap_kg_hr = J_evap_mass * 3600
        
        # Calculate heat carried by evaporation
        q_evap = J_evap_mass * params["h_fg"]
        total_evap_heat += q_evap


        print(f"  Stage {data['stage']}: Evap Temp: {data['T_f']:.2f}°C | Condensation Temp: {data['T_b']:.2f}°C | Vapor Flux: {J_evap_kg_hr:.2f} kg/m²/hr")

    print(f"\nTop Surface Temp: {data["T_top"]:.2f}°C ")
    print("\n--- Overall Performance ---")
    eta_tot = total_evap_heat / params["q_sun"] 
    total_vapor_flux_kg_hr = (total_evap_heat / params["h_fg"]) * 3600
    print(f"Total Vapor Mass Flux: {total_vapor_flux_kg_hr:.2f} kg/m²/hr")
    print(f"Total Efficiency (eta_tot): {eta_tot * 100:.2f}%")

    results = {
        "water productivity (kg/m2/hr)": round(total_vapor_flux_kg_hr, 2),
        "total efficiency (%)": round(eta_tot * 100, 2),
        "absorber temp (C)": round(stage_data[0]['T_f'], 2) if stage_data else None,
        "top surface temp (C)": round(stage_data[0]['T_top'], 2) if stage_data else None,
    }
    return results

# Simulation Cases in Figure 3 

In [ ]:
simulation_cases_Fig3 = [
    {
        "name": "Indoor, windless", #Figure 3b
        "condition": 'indoor',
        "v": 0,
        "a": 0.05,
    },    
    {
        "name": "Indoor, 2 m/s wind", #Figure 3b
        "condition": 'indoor',
        "v": 2.0,
    },
    {
        "name": "Indoor, 4 m/s wind", #Figure 3b
        "condition": 'indoor',
        "v": 4.0,
    },
    {
        "name": "Indoor, 6 m/s wind", #Figure 3b
        "condition": 'indoor',
        "v": 6.0,
    },   
    {
        "name": "Outdoor, windless", #Figure 3c
        "condition": "outdoor",
        "v": 0,
    },       
    {
        "name": "Outdoor, 2 m/s wind", #Figure 3c
        "condition": "outdoor",
        "v": 2.0,
    },      
    {
        "name": "Outdoor, windless, 20°C ambient temperature", #Figure 3e
        "condition": "outdoor",
        "T_b_c": 25,
        "T_amb_c": 20,
        "v": 0, 
    },
    {
        "name": "Outdoor, windless, 30°C ambient temperature", #Figure 3e
        "condition": "outdoor",
        "T_b_c": 35,
        "T_amb_c": 30,
        "v": 0, 
    },
    {
        "name": "Outdoor, windless, 35°C ambient temperature", #Figure 3e
        "condition": "outdoor",
        "T_b_c": 40,
        "T_amb_c": 35,
        "v": 0,
    },
    {
        "name": "Outdoor, windless, 40°C ambient temperature", #Figure 3e
        "condition": "outdoor",
        "T_b_c": 45,
        "T_amb_c": 40,
        "v": 0, 
    },    
    {
        "name": "Outdoor, 0.7 kW/m2 solar flux", #Figure 3f
        "condition": "outdoor",
        "q_sun": 700, 
        "T_b_c": 30,
        "T_amb_c": 25,
        "v": 0,
    },
    {
        "name": "Outdoor, 0.5 kW/m2 solar flux", #Figure 3f
        "condition": "outdoor",
        "q_sun": 500, 
        "T_b_c": 30,
        "T_amb_c": 25,
        "v": 0,
    }, 
    {
        "name": "Outdoor, 0.3 kW/m2 solar flux", #Figure 3f
        "condition": "outdoor",
        "q_sun": 300, 
        "T_b_c": 30,
        "T_amb_c": 25,
        "v": 0,
    },    
]

In [ ]:
# --- Run All Simulation Cases in Fig 3 ---
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
if __name__ == "__main__":
    number_of_stages =  6

    all_results_data = []

    for case in simulation_cases_Fig3:
        print(f"\n========================================================")
        print(f"Running: {case['name']}")
        print(f"========================================================")

        current_params = params.copy()
        current_params.update(case)
        
        current_params["T_b_c"] = current_params["T_amb_c"] + 5
        if current_params["v"] <= 0.1:  # Windless
            current_params["h_a"] = get_h_a_windless(current_params["T_amb_c"], current_params["a"])
        else:  # Windy
            current_params["h_a"] = get_h_a_windy(current_params["T_amb_c"], current_params["v"], current_params["a"])

        # Run Simulation
        output_results = run_multistage_simulation(number_of_stages, current_params)

        result_row = {"name": case["name"]}
        result_row.update(output_results)
        
        all_results_data.append(result_row)
        

    results_df = pd.DataFrame(all_results_data)

    output_filename = os.path.join("results", f"Glass_Fig3_simulation_Fig3_summary_{timestamp}.csv")
    results_df.to_csv(output_filename, index=False)

    print(f"\n\n All Fig 3 simulations complete. Results saved to '{output_filename}'")

# Wind speed vs module dimension (Figure S15)

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
params["condition"] = 'outdoor'
device_widths = [0.05, 0.1, 0.2, 0.5, 1.0, 2.0]  # meters
windspeeds = [0, 2, 4, 6, 8, 10]  # m/s

simulation_cases = [
    {"name": f"a={a:.2f}m_v={v}m/s" if v > 0 else f"a={a:.2f}m_windless",
     "a": a, "v": v}
    for a in device_widths for v in windspeeds]

print(f"Running {len(simulation_cases)} cases...")

if __name__ == "__main__":
    number_of_stages =  6
    all_results_data = []

    for i, case in enumerate(simulation_cases, 1):
        print(f"\n[{i}/{len(simulation_cases)}] {case['name']}")

        current_params = params.copy()
        current_params.update(case)
        current_params["T_b_c"] = current_params["T_amb_c"] + 5  
        
        if current_params["v"] <= 0.1:  
            current_params["h_a"] = get_h_a_windless(current_params["T_amb_c"], current_params["a"])
        else:  
            current_params["h_a"] = get_h_a_windy(current_params["T_amb_c"], current_params["v"], current_params["a"])

        output_results = run_multistage_simulation(number_of_stages, current_params)

        result_row = {"case_number": i, "name": case["name"]}
        result_row.update(output_results)
        all_results_data.append(result_row)
        

    results_df = pd.DataFrame(all_results_data)

    output_filename = os.path.join("results", f"Glass_FigS15c_simulation_summary_{timestamp}.csv")
    results_df.to_csv(output_filename, index=False)

    print(f"\n\n All simulations complete. Results saved to '{output_filename}'")


# T<sub>amb</sub> vs Stage number (Figure S15)

In [ ]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
params["v"] = 0.1
params["condition"] = 'outdoor'
stage_number = [2, 4, 6, 8, 10, 12]
ambient_temp = [15, 20, 25, 30, 35, 40, 45]  # °C

simulation_cases = [
    {"name": f"n={n}_T={T}C", "n_stages": n, "T_amb_c": T}
    for n in stage_number
    for T in ambient_temp]

print(f"Running {len(simulation_cases)} cases...")

if __name__ == "__main__":
    all_results_data = []
    
    for i, case in enumerate(simulation_cases, 1):
        print(f"\n[{i}/{len(simulation_cases)}] {case['name']}")
        
        current_params = params.copy()
        current_params.update(case)
        n_stages = case["n_stages"]
        current_params["T_b_c"] = current_params["T_amb_c"] + 5
        
        if current_params["v"] <= 0.1:
            current_params["h_a"] = get_h_a_windless(current_params["T_amb_c"], current_params["a"])
        else:
            current_params["h_a"] = get_h_a_windy(current_params["T_amb_c"], current_params["v"], current_params["a"])
            
        output_results = run_multistage_simulation(n_stages, current_params)

        result_row = {"case_number": i, "name": case["name"], 
                     "n_stages": n_stages, "T_amb_c": case["T_amb_c"]}
        result_row.update(output_results)
        all_results_data.append(result_row)

    results_df = pd.DataFrame(all_results_data)

    output_filename = os.path.join("results", f"Glass_FigS15d_simulation_summary_{timestamp}.csv")
    results_df.to_csv(output_filename, index=False)

    print(f"\n\n All simulations complete. Results saved to '{output_filename}'")